In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib widget

layout = widgets.Layout(align_items = 'center')

<h1>REGRESIÓN LOGÍSTICA</h1>
<h2>SET DE DATOS</h2>  

In [ ]:
dfHelados = pd.read_csv('helados.csv')
dfHelados.head(5)

Se utilizarán las columnas correspondientes al ingreso de las familias, el precio del helado y la temperatura del día para intentar <strong>predecir si la cantidad de helado consumido fue grande o no</strong>.

<h2>GRADIENTE DESCENDENTE</h2>     
<ol>
    <li>Normalización y escalamiento de las variables de entrada:
         $$
            x_i := \frac{x_i-\mu_i}{\sigma}
         $$
    </li>
</ol>

In [ ]:
col_mask = ['income', 'price', 'temp']
dfHelados2 = dfHelados.loc[:, col_mask]
dfH_norm = (dfHelados2 - dfHelados2.mean(axis = 0)) / dfHelados2.std(axis = 0)  
dfH_norm.head(5)

<ol start="2">
    <li>Conformación de la matriz X a partir de las variables de entrada normalizadas:
         $$
           \begin{bmatrix}
            1\quad x_1^0\quad x_2^0\quad x_3^0\\
            1\quad x_1^1\quad x_2^1\quad x_3^1\\
            ... \\
            1\quad x_1^m\quad x_2^m\quad x_3^m
           \end{bmatrix}
        $$
    </li>
</ol>

In [ ]:
X_test = dfH_norm.values
x0_logR = np.ones(shape = (X_test.shape[0],1))
X_logR = np.hstack((x0_logR, X_test))

<ol start="3">
    <li>Conformación del vector de valores de salida:
        $$
        \begin{bmatrix}
            y^0\\
            y^1\\
            ... \\
            y^m
        \end{bmatrix}
        $$
    </li>
</ol>

In [ ]:
Y_test = dfHelados.loc[:,'bigCons'].values
Y_logR = Y_test.reshape(-1, 1)

<ol start="4">
    <li>Selección del parámetro $\alpha$
    </li>
</ol>

In [ ]:
a_slider_logR = widgets.FloatSlider(min = 0,max = 1,step = 0.001,value = 0.5,description = r'$\alpha$')

<ol start="5">
    <li>Aplicación del algoritmo iterativo de gradiente descendente:
        $$
        \theta_j := \theta_j - \alpha \frac{\partial}{\partial\theta_j}J(\theta_0, \theta_1)
        $$
    </li>
</ol>

In [ ]:
def plot_J_logR(alpha,X,Y):
    m = X.shape[0]
   
    t = np.zeros(shape = (X.shape[1],1))
     
    J = np.array([])

    for i in range(0,200):
        z = X.dot(t[:,i])
        h =  1 / (1 + np.exp(-z))
        h = h.reshape(-1, 1)
        logH = np.log(h) 
        log1_H = np.log(1 - h +  1e-7)  # Para evitar conflictos en el cálculo del logaritmo de 0 se suma un valor pequeño
        J = np.append(J, -1/m * (Y.T.dot(logH) + (1 - Y).T.dot(log1_H)))
        t_temp = np.array([]) 
        for j in range(0,t.shape[0]):
            t_temp = np.append(t_temp, t[j,i] - alpha/m * (X[:,j].reshape(1, -1).dot(h - Y)))
        t = np.hstack((t,t_temp.reshape(-1, 1)))
    return t, J
    

<ol start="6">
    <li>Cálculo de los valores de salida predichos:
        $$
            h_{\theta}(x)=\theta_0x_0+\theta_1x_1+\theta_2x_2+\theta_3x_3
        $$
    </li>
</ol>

In [ ]:
def h_logR_value(t,X):
    z = X.dot(t[:,-1])
    h =  1 / (1 + np.exp(-z))
    h = (h >= 0.5) * 1
    return h

In [ ]:
plots_logR = widgets.Output()

with plots_logR:
    fig_gd, axs_gd = plt.subplots(3,1,figsize = (7,8),tight_layout = True)
    fig_gd.suptitle(r'Gradiente descendente')
    
    axs_gd[1].set_title(fr'Variación de función costo $J(\theta)$')
    t_logR, J_logR = plot_J_logR(a_slider_logR.value,X_logR,Y_logR)
    lineLogR, = axs_gd[1].plot(J_logR)
    axs_gd[1].plot()
    axs_gd[1].set_ylabel(r'$J(\theta)$')   
    axs_gd[1].set_xlabel(r'N° de iteraciones')   
    axs_gd[1].grid(True)
    
    axs_gd[0].set_title(fr'Hipótesis vs Valores de salida reales')
    h_gd = h_logR_value(t_logR,X_logR)
    lineh_logR, = axs_gd[0].plot(h_gd,'rx', label='Consumo predicho',alpha = 0.9)
    axs_gd[0].scatter(np.arange(0,30),Y_logR,marker='o',label='Consumo real',alpha = 0.9)
    axs_gd[0].legend()
    axs_gd[0].grid(True)
                        
    axs_gd[2].set_title(fr'Variación de $\theta_j$')
    theta0, = axs_gd[2].plot(t_logR[0,:],label=r'$\theta_0$')
    theta1, = axs_gd[2].plot(t_logR[1,:],label=r'$\theta_1$')
    theta2, = axs_gd[2].plot(t_logR[2,:],label=r'$\theta_2$')
    theta3, = axs_gd[2].plot(t_logR[3,:],label=r'$\theta_3$')
    axs_gd[2].set_ylabel(r'$\theta$')   
    axs_gd[2].set_xlabel(r'N° de iteraciones')   
    axs_gd[2].grid(True)
    axs_gd[2].legend()

In [ ]:
out_t0_logR = widgets.HTMLMath(value = fr'El valor de $\theta_0$ después de 500 iteraciones es: {t_logR[0,-1]:.4f}')
    
out_t1_logR = widgets.HTMLMath(value = fr'El valor de $\theta_1$ después de 500 iteraciones es: {t_logR[1,-1]:.4f}')

out_t2_logR = widgets.HTMLMath(value = fr'El valor de $\theta_2$ después de 500 iteraciones es: {t_logR[2,-1]:.4f}')
    
out_t3_logR = widgets.HTMLMath(value = fr'El valor de $\theta_3$ después de 500 iteraciones es: {t_logR[3,-1]:.4f}')


In [ ]:
def update_J_logR (change):
    t_logR, J_logR = plot_J_logR(a_slider_logR.value,X_logR,Y_logR)
    lineLogR.set_ydata(J_logR)
    h_logR = h_logR_value(t_logR,X_logR)
    lineh_logR.set_ydata(h_logR)
    theta0.set_ydata(t_logR[0,:])
    theta1.set_ydata(t_logR[1,:])
    theta2.set_ydata(t_logR[2,:])
    theta3.set_ydata(t_logR[3,:])
    out_t0_logR.value = fr'El valor de $\theta_0$ después de 200 iteraciones es: {t_logR[0,-1]:.4f}'
    out_t1_logR.value = fr'El valor de $\theta_1$ después de 200 iteraciones es: {t_logR[1,-1]:.4f}'
    out_t2_logR.value = fr'El valor de $\theta_2$ después de 200 iteraciones es: {t_logR[2,-1]:.4f}'
    out_t3_logR.value = fr'El valor de $\theta_3$ después de 200 iteraciones es: {t_logR[3,-1]:.4f}'
    
a_slider_logR.observe(update_J_logR,'value')

widgets.VBox([plots_logR,a_slider_logR, out_t0_logR, out_t1_logR, out_t2_logR, out_t3_logR],layout=layout)


<div class="alert alert-block alert-success">
<b>Conclusiones:</b>
    <ol>
    <li>Si $\alpha$ es muy pequeño, la convergencia del gradiente 
        descendente es lenta (requiere más iteraciones).</li>
    <li>El valor de la función de costo $J(\theta)$ decrece en 
        cada iteración.</li>
    <li>Mientras menor sea el valor de la función costo $J(\theta)$, 
        la función hipótesis realizará mejores predicciones de los 
        valores de salida $y$.</li>
</ol>
    
</div>

Es posible observar en el siguiente gráfico los resultados de la clasificación
predichos por el modelo entrenado y los valores reales de clasificación:

In [ ]:
X3d = [X_logR[j,1] for j in range(0,Y_logR.shape[0]) if  Y_logR[j] == 1]
Y3d = [X_logR[j,2] for j in range(0,Y_logR.shape[0]) if  Y_logR[j] == 1]
Z3d = [X_logR[j,3] for j in range(0,Y_logR.shape[0]) if  Y_logR[j] == 1]

fig3d, ax3d = plt.subplots(subplot_kw={'projection': '3d'}, tight_layout = True)
ax3d.scatter(X3d, Y3d, Z3d,label='Alto consumo real',s=40)
    
X3d = [X_logR[j,1] for j in range(0,Y_logR.shape[0]) if  Y_logR[j] == 0]
Y3d = [X_logR[j,2] for j in range(0,Y_logR.shape[0]) if  Y_logR[j] == 0]
Z3d = [X_logR[j,3] for j in range(0,Y_logR.shape[0]) if  Y_logR[j] == 0]

ax3d.scatter(X3d, Y3d, Z3d,label='Bajo consumo real',s=40)

t_logR, J_logR = plot_J_logR(a_slider_logR.value,X_logR,Y_logR)
h_logR = h_logR_value(t_logR,X_logR)

X3d = [X_logR[j,1] for j in range(0,h_logR.shape[0]) if  h_logR[j] == 1]
Y3d = [X_logR[j,2] for j in range(0,h_logR.shape[0]) if  h_logR[j] == 1]
Z3d = [X_logR[j,3] for j in range(0,h_logR.shape[0]) if  h_logR[j] == 1]
ax3d.scatter(X3d, Y3d, Z3d,label='Alto consumo predicho',s=100,facecolors='none', edgecolors='g')

X3d = [X_logR[j,1] for j in range(0,h_logR.shape[0]) if  h_logR[j] == 0]
Y3d = [X_logR[j,2] for j in range(0,h_logR.shape[0]) if  h_logR[j] == 0]
Z3d = [X_logR[j,3] for j in range(0,h_logR.shape[0]) if  h_logR[j] == 0]
ax3d.scatter(X3d, Y3d, Z3d,label='Bajo consumo predicho',s=100,facecolors='none', edgecolors='r')

ax3d.legend()
ax3d.set_title(f'Clasificación predicha vs real, alpha= {a_slider_logR.value}')
ax3d.set_xlabel('Income')
ax3d.set_ylabel('Price')   
ax3d.set_zlabel('Temperature') ;

<h3>Gradiente descendente sin normalizar</h3>    

In [ ]:
dfHelados2.head(5)

In [ ]:
X_gdSn = dfHelados2.values
x0_gdSn = np.ones(shape = (X_gdSn.shape[0],1))
X_gdSn = np.hstack((x0_gdSn, X_gdSn))

Y_gdSn = dfHelados.loc[:,'bigCons'].values.reshape(-1, 1)

In [ ]:
alphaSn_slider = widgets.FloatSlider(min = 0,max = 0.09,step = 0.001,value = 0.05,description = r'$\alpha$')

In [ ]:
plots_logR_gdSn = widgets.Output()

with plots_logR_gdSn:
    fig_gdSn, axs_gdSn = plt.subplots(3,1,figsize = (7,8),tight_layout = True)
    fig_gdSn.suptitle(r'Gradiente descendente')
    
    axs_gdSn[1].set_title(fr'Variación de función costo $J(\theta)$')
    t_logR_gdSn, J_logR_gdSn = plot_J_logR(alphaSn_slider.value,X_gdSn,Y_gdSn)
    lineLogR_gdSn, = axs_gdSn[1].plot(J_logR_gdSn)
    axs_gdSn[1].plot()
    axs_gdSn[1].set_ylabel(r'$J(\theta)$')   
    axs_gdSn[1].set_xlabel(r'N° de iteraciones')   
    axs_gdSn[1].grid(True)
    
    axs_gdSn[0].set_title(fr'Hipótesis vs Valores de salida reales')
    h_gdSn = h_logR_value(t_logR_gdSn,X_gdSn)
    lineh_logR_gdSn, = axs_gdSn[0].plot(h_gdSn,'rx', label='Consumo predicho',alpha = 0.9)
    axs_gdSn[0].scatter(np.arange(0,30),Y_gdSn,marker='o',label='Consumo real',alpha = 0.9)
    axs_gdSn[0].legend()
    axs_gdSn[0].grid(True)
                        
    axs_gdSn[2].set_title(fr'Variación de $\theta_j$')
    theta0Sn, = axs_gdSn[2].plot(t_logR_gdSn[0,:],label=r'$\theta_0$')
    theta1Sn, = axs_gdSn[2].plot(t_logR_gdSn[1,:],label=r'$\theta_1$')
    theta2Sn, = axs_gdSn[2].plot(t_logR_gdSn[2,:],label=r'$\theta_2$')
    theta3Sn, = axs_gdSn[2].plot(t_logR_gdSn[3,:],label=r'$\theta_3$')
    axs_gdSn[2].set_ylabel(r'$\theta$')   
    axs_gdSn[2].set_xlabel(r'N° de iteraciones')   
    axs_gdSn[2].grid(True)
    axs_gdSn[2].legend()

In [ ]:
out_t0_gdSn = widgets.HTMLMath(value = fr'El valor de $\theta_0$ después de 10 iteraciones es: {t_logR_gdSn[0,-1]:.4e}')
    
out_t1_gdSn = widgets.HTMLMath(value = fr'El valor de $\theta_1$ después de 10 iteraciones es: {t_logR_gdSn[1,-1]:.4e}')

out_t2_gdSn = widgets.HTMLMath(value = fr'El valor de $\theta_2$ después de 10 iteraciones es: {t_logR_gdSn[2,-1]:.4e}')
    
out_t3_gdSn = widgets.HTMLMath(value = fr'El valor de $\theta_3$ después de 10 iteraciones es: {t_logR_gdSn[3,-1]:.4e}')

In [ ]:
def update_gdSn (change):
    t_logR_gdSn, J_logR_gdSn = plot_J_logR(alphaSn_slider.value,X_gdSn,Y_gdSn)
    axs_gdSn[1].cla()
    lineLogR_gdSn, = axs_gdSn[1].plot(J_logR_gdSn)
    axs_gdSn[1].set_ylabel(r'$J(\theta)$')   
    axs_gdSn[1].set_xlabel(r'N° de iteraciones')  
    axs_gdSn[1].grid(True)
    
    h_gdSn = h_logR_value(t_logR_gdSn,X_gdSn)
    axs_gdSn[0].cla()
    lineh_logR_gdSn, = axs_gdSn[0].plot(h_gdSn,'rx', label='Consumo predicho',alpha = 0.9)
    axs_gdSn[0].scatter(np.arange(0,30),Y_gdSn,marker='o',label='Consumo real',alpha = 0.9)
    axs_gdSn[0].legend()
    axs_gdSn[0].grid(True)
    
    axs_gdSn[2].cla()
    theta0Sn, = axs_gdSn[2].plot(t_logR_gdSn[0,:],label=r'$\theta_0$')
    theta1Sn, = axs_gdSn[2].plot(t_logR_gdSn[1,:],label=r'$\theta_1$')
    theta2Sn, = axs_gdSn[2].plot(t_logR_gdSn[2,:],label=r'$\theta_2$')
    theta3Sn, = axs_gdSn[2].plot(t_logR_gdSn[3,:],label=r'$\theta_3$')
    axs_gdSn[2].set_ylabel(r'$\theta$')   
    axs_gdSn[2].set_xlabel(r'N° de iteraciones')   
    axs_gdSn[2].grid(True)
    axs_gdSn[2].legend();

    out_t0_gdSn.value = fr'El valor de $\theta_0$ después de 10 iteraciones es: {t_logR_gdSn[0,-1]:.4e}'
    out_t1_gdSn.value = fr'El valor de $\theta_1$ después de 10 iteraciones es: {t_logR_gdSn[1,-1]:.4e}'
    out_t2_gdSn.value = fr'El valor de $\theta_2$ después de 10 iteraciones es: {t_logR_gdSn[2,-1]:.4e}'
    out_t3_gdSn.value = fr'El valor de $\theta_3$ después de 10 iteraciones es: {t_logR_gdSn[3,-1]:.4e}'
    
alphaSn_slider.observe(update_gdSn ,'value')

widgets.VBox([plots_logR_gdSn,alphaSn_slider, out_t0_gdSn, out_t1_gdSn, out_t2_gdSn, out_t3_gdSn],layout=layout)


<div class="alert alert-block alert-danger">
<b>Conclusiones:</b> 
    <ol>
        <li>Si las variables de entrada (features) $x_j$ poseen rangos de valores muy diferentes, 
            es <b>necesario</b> realizar la normalización de las mismas para asegurar la 
            convergencia del gradiente descendente.</li>
    </ol>
</div>

<h3>Gradiente descendente con regularización</h3>    

In [ ]:
df_util = dfH_norm.loc[:,['income','price','temp']] 

In [ ]:
extra1 = df_util['temp'].values * 2 + np.random.randn(df_util.shape[0]) 
df_extra1 = pd.DataFrame(extra1)
df_extra1 = (df_extra1 - df_extra1.mean(axis = 0)) / df_extra1.std(axis = 0)  


In [ ]:
dfH_reg = pd.concat([df_util, df_extra1],axis=1)
dfH_reg.head(5)

In [ ]:
dfH_reg.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
X_reg = dfH_reg.values
x0_reg = np.ones(shape = (X_reg.shape[0],1))
X_reg = np.hstack((x0_reg, X_reg))

In [ ]:
Y_reg = dfHelados.loc[:,'bigCons'].values.reshape(-1, 1)
Y_reg = Y_reg

In [ ]:
alphaReg_slider = widgets.FloatSlider(min = 0,max = 0.9,step = 0.01,value = 0.5,description = r'$\alpha$')
lambdaReg_slider = widgets.FloatSlider(min = 0,max = 10.00,step = 0.1,value = 0.00,description = r'$\lambda$')

In [ ]:
def plot_J_reg(alpha, X, Y, l):
    m = X.shape[0]
   
    t = np.zeros(shape = (X.shape[1],1))
     
    J = np.array([])

    for i in range(0,200):
        z = X.dot(t[:,i])
        h =  1 / (1 + np.exp(-z))
        h = h.reshape(-1, 1)
        
        logH = np.log(h) 
        log1_H = np.log(1 - h +  1e-7)
        
        J = np.append(J, -1/m * (Y.T.dot(logH) + (1 - Y).T.dot(log1_H)) +  l/(2*m) * sum(np.square(t[1:,i])))
        
        t_temp = np.array([]) 
        
        for j in range(0,t.shape[0]):
            if j == 0:
                t_temp = np.append(t_temp, t[j,i] - alpha/m * (X[:,j].reshape(1, -1).dot(h - Y)))
            else:
                t_temp = np.append(t_temp, t[j,i] - alpha/m * (X[:,j].reshape(1, -1).dot(h - Y) + l * t[j,i]))
        t = np.hstack((t,t_temp.reshape(-1, 1)))
    return t, J
    

In [ ]:
plots_gdReg = widgets.Output()

with plots_gdReg:
    fig_gdReg, axs_gdReg = plt.subplots(2,1,figsize = (7,6),tight_layout = True)
    fig_gdReg.suptitle(r'Gradiente descendente con regularización')
    
    axs_gdReg[1].set_title(r'Variación de función costo $J(\theta)$')
    t_gdReg, J_gdReg = plot_J_reg(alphaReg_slider.value,X_reg,Y_reg,lambdaReg_slider.value)
    linejReg, = axs_gdReg[1].plot(J_gdReg)
    axs_gdReg[1].set_ylabel(r'$J(\theta)$')   
    axs_gdReg[1].set_xlabel(r'N° de iteraciones')   
    axs_gdReg[1].grid(True)
    
    axs_gdReg[0].set_title(r'Hipótesis vs Valores de salida reales')
    h_gdReg = h_logR_value(t_gdReg, X_reg)
    linehReg, = axs_gdReg[0].plot(h_gdReg,'rx', label='Consumo predicho',alpha = 0.9)
    axs_gdReg[0].plot(Y_reg,'o',label='Consumo real',alpha = 0.9)
    axs_gdReg[0].legend()
    axs_gdReg[0].grid(True)
    

In [ ]:
out_J = widgets.HTMLMath(value = fr'El valor de $J(\theta)$  después de 200 iteraciones es: {J_gdReg[-1]:.5f}')

In [ ]:
def update_gdReg (change):
    t_gdReg, J_gdReg = plot_J_reg(alphaReg_slider.value,X_reg,Y_reg,lambdaReg_slider.value)
    linejReg.set_ydata(J_gdReg)
    h_gdReg = h_logR_value(t_gdReg,X_reg)
    linehReg.set_ydata(h_gdReg)
    
    out_J.value = fr'El valor de $J(\theta)$ después de 200 iteraciones es: {J_gdReg[-1]:.5f}'
    
alphaReg_slider.observe(update_gdReg ,'value')
lambdaReg_slider.observe(update_gdReg ,'value')

widgets.VBox([plots_gdReg,alphaReg_slider,\
              lambdaReg_slider, \
              out_J], layout=layout)


<div  class="alert alert-block alert-warning">
<b>Conclusiones:</b> 
    <ol>
        <li>Cuando existe una fuerte correlación entre 2 variables de entrada, es necesario aplicar regularización 
            para simplificar el modelo.</li>
        <li>Una variable correlacionada a otra no tendrá un gran impacto en mejorar el modelo, solo le 
            agregará complejidad.</li>
        <li>Aplicar regularización evita problemas de overfitting.</li>
        <li>El parámetro $\lambda$ es un número real positivo que controla el impacto de la regularización.</li>
        <li>Al aumentar el grado de regularización, aumenta el valor de la función de costo $J(\theta)$.</li>
    </ol>
</div>

<h2>IMPLEMENTACIÓN CON LIBRERÍAS ESPECIALIZADAS</h2>
<ol>
    <li>Se utiliza la librería <strong>scikit-learn</strong>
    </li>
</ol>

In [ ]:
from sklearn.linear_model import LogisticRegression

<ol start="2">
    <li>Los arreglos de entrada y salida deben ser <strong>NumPy arrays</strong>.
        <ul>
            <li>El arreglo de <strong>X</strong> debe tener <strong>2 dimensiones</strong>: cada columna debe corresponder con una variable de entrada y cada fila con una observación particular.</li>
            <li>El arreglo de <strong>Y</strong> debe tener <strong>1 dimensión</strong>, siendo cada elemento la salida de cada una de las observaciones</li>
        </ul>
    </li>
</ol>

In [ ]:
X_test

In [ ]:
Y_test

<ol start="3">
    <li>Se crea una instancia de la clase <strong>LogisticRegression</strong>, la cual tiene muchos parámetros opcionales:
        <ul>
          <li><strong>penalty</strong> permite seleccionar si se realiza la regularización y que tipo utilizar: 'l2','l1', 'elasticnet', 'none'.</li>
          <li><strong>C</strong> es un numero que define la potencia de la regularización, valores pequeños indican mayor regularización.</li>
          <li><strong>solver</strong> define que método utilizar para entrnar el modelo: 'liblinear','newton-cg', 'lbfgs', 'sag','saga'.</li>
          <li><strong>max_iter</strong> define el número máximo de iteraciones realizadas al ajustar el modelo.</li>
        </ul>
        Por defecto se utiliza el método 'liblinear', el cual no funciona sin regularización, por lo que se le asigna un valor muy grande a c para disminuir la regularización.
    </li>
</ol>

In [ ]:
model = LogisticRegression(C=1e10,max_iter=500)

<ol start="4">
    <li>Se entrena el modelo, lo cual determina los valores de los coeficientes que corresponden al mejor valor de la función costo
    </li>
</ol>

In [ ]:
model.fit(X_test, Y_test)

<ol start="5">
    <li>Se pueden obtener los atributos del modelo:
        <ul>
            <li> Los distintos valores posibles de salida.
            </li>
        </ul>
    </li>
</ol>

In [ ]:
model.classes_

<ol style="list-style-type:none;">
    <li>
        <ul>
            <li> Los valores de los coeficientes del modelo entrenado.
            </li>
        </ul>
    </li>
</ol>

In [ ]:
print(model.intercept_,model.coef_)

<ol start="6">
    <li>Se calculan los valores predichos de salida haciendo uso del modelo entrenado.
    </li>
</ol>

In [ ]:
predictions = model.predict(X_test)
predictions

In [ ]:
skl = widgets.Output()

with skl:
    fig_p, ax_p = plt.subplots(figsize = (6,4))
    fig_p.suptitle(r'Hipótesis vs Valores de salida reales')
    ax_p.plot(np.arange(0,len(Y_logR)), predictions, 'b+',markersize = 8, label = 'Predicción con sklearn', alpha = 0.8)
    ax_p.scatter(np.arange(0,len(Y_logR)), Y_logR, marker = 'o', s = 25, label='Salida real', alpha = 0.9, facecolors = 'g', edgecolors = 'g')
    lineh_skl, = ax_p.plot(np.arange(0,len(Y_logR)), h_logR,'rx', markersize = 8, label = 'Predicción', alpha = 0.8)
    ax_p.grid(True)
    ax_p.legend();
    
def update_skl (change):
    t_logR, J_logR = plot_J_logR(a_slider_logR.value,X_logR,Y_logR)
    h_logR = h_logR_value(t_logR,X_logR)
    lineh_skl.set_ydata(h_logR)

a_slider_logR.observe(update_skl ,'value')

widgets.VBox([skl,a_slider_logR],layout = layout)

<ol start="7">
    <li>Se calcula un puntaje para el modelo, el mismo equivale al porcentaje de predicciones correctas realizadas por el modelo.
    </li>
</ol>

In [ ]:
model.score(X_test, Y_test)